In [15]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the dataset
df = pd.read_csv(r'C:\Users\KIIT\Desktop\8th sem project\student-por.csv')

# Selecting relevant features including G1 and G2
features = ['G1', 'G2', 'age', 'Medu', 'Fedu', 'studytime', 'failures', 
            'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
target = 'G3'

# Encode categorical features
categorical_features = ['schoolsup', 'famsup', 'higher']
for col in categorical_features:
    if col in df.columns:
        df[col] = df[col].map({'yes': 1, 'no': 0})

X = df[features]
y = df[target]

# Normalize numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Reshape input data for CNN (adding a dimension)
X_scaled = X_scaled.reshape(-1, X_scaled.shape[1], 1)

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Build the CNN model
model = keras.Sequential([
    keras.layers.Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    keras.layers.MaxPooling1D(pool_size=2),
    keras.layers.Conv1D(32, kernel_size=3, activation='relu'),
    keras.layers.MaxPooling1D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(1, activation='linear')  # Regression output
])

# Compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='huber_loss', metrics=['mae'])

# Early stopping to prevent overfitting
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=150, batch_size=16, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f'Mean Absolute Error on test data: {mae}')

# Function for predicting student performance
def predict_performance(new_data):
    new_data_df = pd.DataFrame([new_data], columns=features)
    new_data_scaled = scaler.transform(new_data_df).reshape(-1, len(features), 1)
    prediction = model.predict(new_data_scaled)
    return round(prediction[0][0])

# Example prediction (ensure G1 and G2 are provided)
new_student = [13, 1, 18, 4, 4, 2, 0, 3, 3, 1, 1, 5, 4]
predicted_score = predict_performance(new_student)
print(f'Predicted Final Grade (G3): {predicted_score}')


Epoch 1/150
33/33 [==============================] - 2s 15ms/step - loss: 9.8107 - mae: 10.3081 - val_loss: 6.8220 - val_mae: 7.3151
Epoch 2/150
33/33 [==============================] - 0s 6ms/step - loss: 3.9409 - mae: 4.4144 - val_loss: 2.4114 - val_mae: 2.8746
Epoch 3/150
33/33 [==============================] - 0s 6ms/step - loss: 2.9207 - mae: 3.3801 - val_loss: 1.9415 - val_mae: 2.3871
Epoch 4/150
33/33 [==============================] - 0s 6ms/step - loss: 2.6209 - mae: 3.0803 - val_loss: 1.7083 - val_mae: 2.1549
Epoch 5/150
33/33 [==============================] - 0s 5ms/step - loss: 2.4141 - mae: 2.8685 - val_loss: 1.5795 - val_mae: 2.0296
Epoch 6/150
33/33 [==============================] - 0s 5ms/step - loss: 2.0437 - mae: 2.4913 - val_loss: 1.4647 - val_mae: 1.9053
Epoch 7/150
33/33 [==============================] - 0s 6ms/step - loss: 2.1394 - mae: 2.5977 - val_loss: 1.4080 - val_mae: 1.8500
Epoch 8/150
33/33 [==============================] - 0s 5ms/step - loss: 1.9828 -